In [1]:
import csv
import re
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def readCSV(csv_path, skip_firstRow=False, separator=',', encoding="utf-8-sig"):
    data = []
    with open(csv_path, 'rt', encoding=encoding) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=separator)
        for row in csvreader:
            if not skip_firstRow:
                data.append(row)
            skip_firstRow = False
    return data

In [3]:
list_one = readCSV('lungcheck_200813_drink.csv',skip_firstRow=True)

In [4]:
list_one[:10]

[['27357648',
  '20170214',
  '79',
  '-',
  '',
  '맥주',
  '1',
  '79',
  '',
  '0.5',
  '',
  '1',
  '',
  '4'],
 ['27377046',
  '20170307',
  '75',
  '10년간',
  '10',
  '소주',
  '2',
  '65',
  '65',
  '3-5잔',
  '',
  '2회/주',
  '8',
  '1'],
 ['27392425',
  '20170226',
  '67',
  '40년',
  '40',
  '막걸리,맥주,소주',
  '1,2,4',
  '27',
  '27',
  '3-5잔',
  '',
  '2',
  '',
  '2'],
 ['27392425',
  '20180318',
  '68',
  '40년',
  '40',
  '',
  '',
  '28',
  '28',
  '3-5잔',
  '',
  '주 2회 ',
  '8',
  '3'],
 ['27393101',
  '20180703',
  '44',
  '20년',
  '20',
  '',
  '',
  '24',
  '24',
  '2병',
  '2',
  '2회',
  '',
  '3'],
 ['27396737',
  '20170301',
  '52',
  '10년간 ',
  '10',
  '소주',
  '2',
  '42',
  '42',
  '3~5잔/일 ',
  '',
  '1회/주 ',
  '4',
  '1'],
 ['27404892',
  '20170628',
  '44',
  '25년',
  '25',
  '맥주',
  '1',
  '19',
  '19',
  '6~10잔',
  '',
  '2/주',
  '8',
  '1'],
 ['27404892',
  '20170711',
  '44',
  '25',
  '25',
  '맥주',
  '1',
  '19',
  '19',
  '06월 10일',
  '',
  '2',
  '',
  '2'],
 ['27425

In [5]:
x=[]
xmax=0
for i in list_one:
    x.append(i[11])
    xmax=max(xmax,len(i[11]))
xmax

19

In [6]:
y=[]
ymax=0
for i in list_one:
    try:
        y.append(float(i[12]))
        ymax=max(ymax,float(i[12]))
    except:
        y.append(0)
ymax=int(ymax)

In [7]:
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(x)

In [8]:
input_dim= len(src_tokenizer.word_index) + 1

In [9]:
x_train_real, x_test_real, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False, random_state=1004)

In [10]:
len(y_train)

3594

In [11]:
x_train=src_tokenizer.texts_to_sequences(x_train_real)
x_test=src_tokenizer.texts_to_sequences(x_test_real)

In [12]:
x_test_real[:10]

['1', '2회/주', '3회/주', '2회/주', '3회/일', '6회/wks', '6회/wks', '매일', '1', '1회/월']

In [13]:
x_test[:10]

[[3], [4, 1], [5, 1], [4, 1], [5, 23], [19, 50], [19, 50], [13], [3], [2, 21]]

In [14]:
for i in x_train:
    xmax_2=max(xmax,len(i))
xmax_2

19

In [15]:
index_to_word = src_tokenizer.index_word

In [16]:
x_train = pad_sequences(x_train, padding='post', maxlen=xmax_2)
x_test = pad_sequences(x_test, padding='post', maxlen=xmax_2)

In [17]:
y_train = to_categorical(y_train,num_classes=ymax+1)
y_test = to_categorical(y_test,num_classes=ymax+1)

In [18]:
x_train.shape

(3594, 19)

In [19]:
x_train[0]

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
y_train.shape

(3594, 41)

In [21]:
y_train[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [22]:
words_input = tf.keras.layers.Input(shape=(xmax_2, ),dtype='int32', name='modelInput')
words_input

<tf.Tensor 'modelInput:0' shape=(None, 19) dtype=int32>

In [23]:
words = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=128, mask_zero=True)(words_input)
words

<tf.Tensor 'embedding/Identity:0' shape=(None, 19, 128) dtype=float32>

In [24]:
ff = tf.keras.layers.Flatten()(words)

In [25]:
denseLayer = tf.keras.layers.Dense(ymax+1, activation='softmax')(ff)

In [26]:
modelFunc = tf.keras.Model(inputs=words_input, outputs=denseLayer)

In [27]:
modelFunc.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelFunc.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
modelInput (InputLayer)      [(None, 19)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 19, 128)           37376     
_________________________________________________________________
flatten (Flatten)            (None, 2432)              0         
_________________________________________________________________
dense (Dense)                (None, 41)                99753     
Total params: 137,129
Trainable params: 137,129
Non-trainable params: 0
_________________________________________________________________


In [28]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [35]:
modelFunc.fit(x_train, y_train, batch_size=128, epochs=20)

Train on 3594 samples
Epoch 1/20
3594/3594 [==============================] - 0s 47us/sample - loss: 0.0664 - accuracy: 0.9903
Epoch 2/20
3594/3594 [==============================] - 0s 42us/sample - loss: 0.0611 - accuracy: 0.9905
Epoch 3/20
3594/3594 [==============================] - 0s 42us/sample - loss: 0.0568 - accuracy: 0.9908
Epoch 4/20
3594/3594 [==============================] - 0s 42us/sample - loss: 0.0529 - accuracy: 0.9925
Epoch 5/20
3594/3594 [==============================] - 0s 42us/sample - loss: 0.0486 - accuracy: 0.9922
Epoch 6/20
3594/3594 [==============================] - 0s 41us/sample - loss: 0.0454 - accuracy: 0.9930
Epoch 7/20
3594/3594 [==============================] - 0s 41us/sample - loss: 0.0422 - accuracy: 0.9947
Epoch 8/20
3594/3594 [==============================] - 0s 41us/sample - loss: 0.0395 - accuracy: 0.9964
Epoch 9/20
3594/3594 [==============================] - 0s 45us/sample - loss: 0.0374 - accuracy: 0.9967
Epoch 10/20
3594/3594 [==========

In [36]:
modelFunc.evaluate(x_test, y_test,verbose=0)

[0.15987454906047371, 0.96755356]

In [37]:
x_test[0]

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
predictions=modelFunc.predict(x_test)

In [39]:
for i in range(len(x_test)):
    
    if np.argmax(y_test[i]) != np.argmax(predictions[i]):
        print(x_test_real[i])
        print('실제값: {} 예측값: {}'.format(np.argmax(y_test[i]), np.argmax(predictions[i])))


05월 07일
실제값: 20 예측값: 0
일주일에 다섯번정도
실제값: 20 예측값: 4
5-7일/주
실제값: 28 예측값: 20
2주일에 1회
실제값: 2 예측값: 0
2주회 1회
실제값: 2 예측값: 0
1회/1월
실제값: 1 예측값: 0
일주일3회
실제값: 12 예측값: 0
5회/1week
실제값: 20 예측값: 0
6일/주
실제값: 24 예측값: 4
1회/2주일
실제값: 2 예측값: 0
3주일에 1회
실제값: 1 예측값: 0
05월 07일
실제값: 20 예측값: 0
05월 07일
실제값: 20 예측값: 0
05월 07일
실제값: 20 예측값: 0
3병/1주일
실제값: 12 예측값: 4
거의 2-3잔
실제값: 0 예측값: 28
2달에 1회
실제값: 0 예측값: 4
1/1W
실제값: 4 예측값: 0
6번/1주일
실제값: 24 예측값: 4
반주로 일주일에 7회
실제값: 28 예측값: 8
4회/1week
실제값: 16 예측값: 0
3-4병/1달
실제값: 4 예측값: 3
0.3회/주 
실제값: 1 예측값: 12
05월 07일
실제값: 20 예측값: 0
05월 07일
실제값: 20 예측값: 0
1/1회
실제값: 0 예측값: 4
한달 2~3회
실제값: 3 예측값: 12
1회/2주일
실제값: 2 예측값: 0
한달에2번
실제값: 2 예측값: 0
2~3/7
실제값: 12 예측값: 0
05월 07일
실제값: 20 예측값: 0
일주일에 4회정도
실제값: 16 예측값: 4
한달에 4~5회
실제값: 5 예측값: 2
1회.주
실제값: 0 예측값: 4
한달에1~2번
실제값: 2 예측값: 0
한달2~3회
실제값: 3 예측값: 0
월1~2회
실제값: 2 예측값: 0
3-4번/ 1달 
실제값: 4 예측값: 16
3회정도/일주
실제값: 12 예측값: 0
2~3회/2주
실제값: 6 예측값: 3
한달에 4회
실제값: 4 예측값: 1
한달에 서너번
실제값: 0 예측값: 1
2회~3회/1주일
실제값: 12 예측값: 8
12회/1주일
실제값: 12 예측값: 4
2회/7
실제값: 8 예측값: 0
일주

In [ ]:
inx=2111

In [ ]:

print('예측값: {} 실제값: {}'.format(np.argmax(y_test[inx]), np.argmax(predictions[inx])))